# Ephemerides

The ephemerides are the position end velocity Cartesian vectors of a moving object. In `pykep` the class {py:class}`~pykep.planet` offers a common interface to access various ephemerides, regardless on how they are computed. Whether the underlying computations are simply Keplerian, or based on more advanced trajectory propagations, interpolations or predictions, a unified interface is offered by this type-erasing class.

The user can code his own python class following the mandatory interface of a {py:class}`~pykep.planet` and his own coded objects will be treated uniformly as any other planet in `pykep`. These User Defined Planets (or UPLAs) can implement heterogeneous techniques and interface to any third-party API, but once they are used to construct a {py:class}`~pykep.planet`, they will appear as any other {py:class}`~pykep.planet` in `pykep`.

For convenience, a number of already coded UDPLAs are offered so that, to start with, users can compute the positions of planet, satellites comets and spacecarft without the need to code their own UDPLA.

In [1]:
import numpy as np
import pykep as pk

# UDPLAs in pykep

Let us start to use the UDPLA {py:class}`~pykep.udpla.keplerian` which describes the motion of an object in a Keplerian orbit. All UDPLAS provided by `pykep` are in the module {py:mod}`~pykep.udpla`

In [2]:
# We instantiate the udpla
udpla = pk.udpla.keplerian(pk.epoch(0.), [1., 0, 0, 0, 0, 0], mu_central_body = 1., name = "A Circular Orbit")
# And use it to construct a pykep planet, hence erasing its type.
pla = pk.planet(udpla)

```{note}
Non-dimensional units are often useful in astrodynamics and most pykep class allow their use. Of course only if everything is consistent the computations will make sense. `pykep` does not check this consistency and thus the user must be careful when using anything which is non SI, and to not mix units.
```

Let us print on screen this {py:class}`~pykep.planet`.

In [3]:
print("Short repr: \n", pla)
print("Long repr: \n", pla.extra_info)

Short repr: 
 A Circular Orbit
Long repr: 
 Keplerian planet elements: 
Semi major axis: 1
Semi major axis (AU): 6.684587122268445e-12
Eccentricity: 0
Inclination (deg.): 0
Big Omega (deg.): 0
Small omega (deg.): 0
True anomly (deg.): 0
Mean anomly (deg.): 0
Elements reference epoch (MJD2000): 0
Elements reference epoch (UTC): 2000-01-01T00:00:00.000000
r at ref. = [1, 0, 0]
v at ref. = [0, 1, 0]



The textual representation of a planet can be extracted in two parts.

The first part is common to all objects of type {py:class}`~pykep.planet` and reports the body name only.

The second part is instead original with the udpla type, and is essentially whatever is returned by the {py:class}`~pykep.planet` interface optional method `extra_info`. In this case, the udpla is a c++ based class exposed to python as :class:`pykep.udpla.keplerian` and its extra info report the orbital parameters as well as the Cartesian components of the body state at the reference epoch.

# Custom UDPLAs

Lets see how users can define their own custom udplas and thus leverage the common {py:class}`~pykep.planet` interface. 
A minimal version includes the method *eph* only. Here we will code a very simple and strange object moving in a rectilinear constant motion
along the x-axis. Its a ficticious object and it only serves the purpose of this tutorial.

In [4]:
class my_udpla:
    # This is the only mandatory method in the planet interface
    def eph(self, mjd2000):
        return [mjd2000*np.array([1., 0., 0.]), np.array([1., 0., 0.])]

# Lets instantiate a planet
pla = pk.planet(my_udpla())

# And test
print("call to *eph* works, calls user implementation:", pla.eph(2.))
print("call to *eph_v* also works, calls default implementation:\n", pla.eph_v([1., 2.]))
print("call to *mu_central_body* returns a default -1 value signalling its not implemented:", pla.mu_central_body)
print("call to *name* returns a default value with little info:", pla.name)
# etc....

call to *eph* works, calls user implementation: [[2.0, 0.0, 0.0], [1.0, 0.0, 0.0]]
call to *eph_v* also works, calls default implementation:
 [[ 1.  0.  0.  1.  0.  0.]
 [ 2.  0.  0.  1.  0.  0.]]
call to *mu_central_body* returns a default -1 value signalling its not implemented: -1.0
call to *name* returns a default value with little info: <class '__main__.my_udpla'>


We can then add a bunch of optional method unlocking various functionalities. A rather complete final udpla could look like this:

In [5]:
class my_udpla:
    # This is the only mandatory method in the planet interface
    def eph(self, mjd2000):
        return [mjd2000*np.array([1., 0., 0.]), np.array([1., 0., 0.])]

    # Often used to compute gradients in some optimal control, the acc method (optional)
    # returns a (acc), while eph are r,v (pos, vel). Its default implementation
    # will return -mu/r^3 if mu is present, or throw otherwise.
    def acc(self, mjd2000):
        return np.array([4., 5., 4.])*mjd2000
    
    # Vectorized versions for eph and acc have default naive implementations, which can be (optionally) overidden.
    def acc_v(self, mjd2000s):
        # We implement a naive vectorization
        retval = np.zeros((len(mjd2000s), 3))
        for i, mjd2000 in enumerate(mjd2000s):
            acc = self.acc(mjd2000)
            retval[i,:3] = acc
        return retval
    def eph_v(self, mjd2000s):
        # We implement a naive vectorization
        retval = np.zeros((len(mjd2000s), 6))
        for i, mjd2000 in enumerate(mjd2000s):
            r, v = self.eph(mjd2000)
            retval[i,:3] = r
            retval[i,3:] = v
        return retval
    
    # A bunch of methods allow to add information to
    # the udpla (and exposed in the planet interface) which can be used in some contexts and 
    # also affect the availability of default implementations of, for example, acc and elements.
    def get_mu_central_body(self):
        return 3.
    
    def get_mu_self(self):
        return 1.1
    
    def get_radius(self):
        return 0.345
    
    def get_safe_radius(self):
        return 0.124
    
    def get_name(self):
        return self.whoami
    
    def get_extra_info(self):
        return "Extra info"
    
    # We can also have a constructor to have some internal state.
    def __init__(self, whoami):
        self.whoami = whoami
    
pla2 = pk.planet(my_udpla("I have all methods! I feel complete."))

In [6]:
print("call to *eph* works, calls user implementation:", pla2.eph(2.))
print("call to *eph_v* calls user implementation:\n", pla2.eph_v([1., 2.]))
print("call to *mu_central_body* calls user implementation:", pla2.mu_central_body)
print("call to *name* calls user implementation:", pla2.name)
print("call to *period* makes use of the user provided mu:", pla2.period(2.))
print("call to *elements* makes use of the user provided mu:", pla2.elements(2., pk.el_type.MEE))

call to *eph* works, calls user implementation: [[2.0, 0.0, 0.0], [1.0, 0.0, 0.0]]
call to *eph_v* calls user implementation:
 [[ 1.  0.  0.  1.  0.  0.]
 [ 2.  0.  0.  1.  0.  0.]]
call to *mu_central_body* calls user implementation: 3.0
call to *name* calls user implementation: I have all methods! I feel complete.
call to *period* makes use of the user provided mu: 6.664324407237548
call to *elements* makes use of the user provided mu: [0.0, nan, nan, nan, nan, nan]


Note the nans deriving from the impossibility to define osculating equinoctial elemnts for this *weird* body moving along a line passing throught eh origin ... all good. Equinoctial elements are non-singular .... up to a certain extent!
